## The Testing phase

### Goal: Using Keras, implement _______________


### Install the following libraries:

pip install numpy

pip install pandas

pip install scikit-learn

pip install tensorflow



In [1]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!

/sfs/weka/scratch/eyf3gu/DS4002/DS4002Project3/SCRIPTS


In [1]:
# install libraries
import os
import numpy as np
import pandas as pd
import tensorflow
from sklearn.model_selection import train_test_split # for splitting data into training / test sets
from tensorflow.keras.preprocessing.image import ImageDataGenerator # 
from tensorflow.keras import layers, models, optimizers

data_dir = "/sfs/weka/scratch/eyf3gu/DS4002/DS4002Project3/DATA/celebrities_all" # set base directory
image_size = (150, 150)  # Resize images to 150 x 150
batch_size = 32 # batch size refers to the number of images that will be processed at a time before the model's parameters are reset. 
# we chose this number becuase it is relatively standard.
epochs = 100 # we settled on 100 epochs for the testing phase

# using ImageDataGenerator, load / preprocess the images
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
# the "rescale" parameter scales down the range of pixel sizes in the images to between 0 and 1. This helps with processing speed.
# "validation-split" defines a 20/80 test/training split (0.2 = ratio of images placed in testing category)

# Training data generator -- method to produce a set of images based on chosen characteristics
train_generator = datagen.flow_from_directory( 
    data_dir, # specifies path to target directory ("celebrities_all")
    target_size=image_size, # standardizes image size to 150 x 150, as specified earlier
    batch_size=batch_size, # batch size is 32, as specified earlier
    class_mode='categorical', # the model will be identifying celeb images, where each celeb is a different "class". Celeb is a categorical variable
    subset='training' # the generator will pull the training images, which make up 80% of the celebrities_all images.
)

# Validation data generator -- repeating the steps used to define the train_generator. Parameters are the same, but the generator will pull from the 20% of images in the test (validation) set
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation' # pulling from the validation set
)

# Now, we will build a simple CNN model using Keras. 
# First, using the models.sequential class, a sequential model will be created to produce a stack of layers. Each layer will have an input
# sensor and an output sensor. This will create a "feed-forward" neural network wherein each layer is directly connected to the one before it.


model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 3. Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# 4. Evaluate the model and store accuracies
accuracy_results = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy_results[1]}")

# Create a DataFrame to store the accuracy of each image
celebs_accuracy = pd.DataFrame({
    "image_path": validation_generator.filenames,
    "accuracy": [accuracy_results[1]] * len(validation_generator.filenames)
})

# Save the DataFrame to a CSV file (optional)
celebs_accuracy.to_csv("celebs_accuracy.csv", index=False)

print("Accuracy data saved to celebs_accuracy.csv.")


2024-11-20 12:20:25.333984: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-20 12:20:25.366415: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-20 12:20:26.927352: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-20 12:20:27.180236: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732123227.663816  523770 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732123227.80

Found 8000 images belonging to 25 classes.
Found 2000 images belonging to 25 classes.


/home/eyf3gu/.local/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-20 12:20:45.368950: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/100


/home/eyf3gu/.local/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 33s 127ms/step - accuracy: 0.0561 - loss: 3.1907 - val_accuracy: 0.1545 - val_loss: 2.8708
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 31s 125ms/step - accuracy: 0.1875 - loss: 2.7512 - val_accuracy: 0.3180 - val_loss: 2.3316
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 32s 126ms/step - accuracy: 0.3560 - loss: 2.1317 - val_accuracy: 0.5060 - val_loss: 1.7718
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 31s 126ms/step - accuracy: 0.5705 - loss: 1.4189 - val_accuracy: 0.6550 - val_loss: 1.3720
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 32s 126ms/step - accuracy: 0.7578 - loss: 0.7953 - val_accuracy: 0.7595 - val_loss: 1.1985
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 32s 126ms/step - accuracy: 0.8984 - loss: 0.3402 - val_accuracy: 0.7825 - val_loss: 1.3186
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 31s 126ms/step - accuracy: 0.9452 - loss: 0.1791 - val_accuracy: 0.7735 - val_loss: 1.7257
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 31s 126ms/step - accuracy: 0.9643 - loss: 0.127